In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np

In [2]:
os.chdir('final')

FileNotFoundError: [WinError 2] Не удается найти указанный файл: 'final'

In [2]:
#Сбор всех данных в одну выборку
#for_train - берет только размеченные данные на агрессивность 
#to_csv - сохраняет в csv

def creadte_dataset(to_csv=False, for_train=False): 
    df = pd.read_csv('df_ride_data.csv')
    df1 = pd.read_csv('df_ride_data_part2.csv')
    df = df.append(df1)
    
    df_mark = pd.read_csv('df_marks.csv')
    df_client_rate = pd.read_csv('df_cl_rates_on_rides.csv')
    df_driver_rate = pd.read_csv('df_dr_rates_on_rides.csv')
    
    df = pd.merge(df, df_mark, on='id_order', how='left')
    df_mark = pd.DataFrame(None)
    
    df = pd.merge(df, df_client_rate, on='id_order', how='left')
    df_client_rate = pd.DataFrame(None)
    
    df = pd.merge(df, df_driver_rate, on='id_order', how='left')
    df_driver_rate = pd.DataFrame(None)
    
    df_prev = pd.read_csv('merged_data_driver.csv')[['id_order', 'last_total_dist', 'last_total_time', 'last_total_cnt', 
                                             'total_dist', 'total_time', 'total_cnt']]
    df = pd.merge(df, df_prev, on='id_order', how='left')
    df_prev = pd.DataFrame(None)
    
    df_prev_cl = pd.read_csv('merged_data_client.csv')[['id_order', 'last_total_dist', 'last_total_time', 'last_total_cnt', 
                                             'total_dist', 'total_time', 'total_cnt']]
    df_prev_cl.columns = ['id_order'] + ['cl_' + i for i in df_prev_cl.columns[1:]]
    df = pd.merge(df, df_prev_cl, on='id_order', how='left')
    df_prev_cl = pd.DataFrame(None)
    
    if for_train:
        df_y = pd.read_csv('df_y.csv')
        df_y.columns = ['id_order', 'is_aggressive', 'comment']
        df = pd.merge(df_y, df, on='id_order', how='inner')
        
        df_y = pd.DataFrame(None)
    
    if to_csv:
        df.to_csv('data.csv')
        
    return df

In [3]:
df_train = creadte_dataset(False, True)

In [4]:
df_train.to_csv('train_v4.csv')

In [5]:
df = creadte_dataset(False, False)

In [6]:
def get_day_week(arr):
    lst = []
    for i in arr:
        lst.append(int(i.weekday()))
    return lst

def get_year(arr):
    lst = []
    for i in arr:
        lst.append(int(i.year))
    return lst

def get_month(arr):
    lst = []
    for i in arr:
        lst.append(int(i.month))
    return lst

def get_minute(arr):
    lst = []
    for i in arr:
        lst.append(i.hour * 60 + i.minute)
    return lst

def get_day(arr):
    lst = []
    for i in arr:
        lst.append(i.timetuple().tm_yday)
    return lst

def get_country_mark(arr):
    lst = [i.split()[0] for i in arr]
    return lst

def comment_cnt(arr):
    lst = []
    for i in arr:
        lst.append(len(all_comments[all_comments.driver_id == i]))
    return lst

def is_nigth(arr):
    lst = []
    for i in arr:
        if i.hour < 6 or i.hour >= 23:
            lst.append(1)
        else:
            lst.append(0)
    return lst

def is_rate(arr): 
    return [int(i != -1) for i in arr]

In [7]:
def prepare_data(data): 
    data['time'] = pd.to_datetime(data.dt_15_min, infer_datetime_format=True)
    
    data['day'] = get_day(data['time'])
    data['minute'] = get_minute(data['time'])
    data['month']= get_month(data['time'])
    data['week_day']= get_day_week(data['time'])
    data['is_nigth']= is_nigth(data['time'])
    
    data['avg_speed'] = data['distance'] / (data['duration'] + 1)
    data['avg_arrived_speed'] = data['arrived_distance'] / (data['arrived_duration'] + 1)
    
    
    data['is_rate'] = is_rate(data['driver_rate_ride'].fillna(-1).values)
    data['driver_rate_ride'] = data['driver_rate_ride'].fillna(4.7)
    data['client_rate_ride'] = data['client_rate_ride'].fillna(5.0)
    data['mark'] = data['mark'].fillna('none')
    
    countries = {"kia" : "Корея", "volkswagen" : "Германия", "mercedesbenz" : "Германия", "hyundai" : "Корея", 
                        "skoda" : "Чехия",
                        "renault" : "Франция", "ford" : "США", "toyota" : "Япония", "bmw" : "Германия", "chevrolet" : "США",
                        "nissan" : "Япония", "daewoo" : "Корея", "changan" : "Китай", "opel" : "Германия", "lada" : "",
                        "ravon" : "Узбекистан", 
                        "lexus" : "Япония", "lifan" : "Китай", "mitsubishi" : "Япония", "lada" : "Россия", "citroen" : "Франция", 
                        "audi" : "Германия", "mazda" : "Япония", "brilliance" : "Китай", "peugeot" : "Франция",
                        "honda" : "Япония", "genesis" : "Корея", "geely" : "Китай", 
                        "datsun" : "Япония", "fiat" : "Италия", "vortex" : "Россия", 
                        "jac" : "Китай", "volvo" : "Швеция", "dongfeng":"Китай", "jeep":"Америка", 
                        "seat":"Испания", "iran kondro": "Иран", "rolls-royce":"Англия", "ваз": "Россия", 
                        "ferrari":"Италия", "lada (ваз)":"Россия", "haima":"Китай", "great wall":"Китай", "infiniti":"Япония", "GAC":"Китай", "smart":"Германия", 
                         "suzuki":"Япония", "saab":"швеция", "bentley":"Англия", "pontiac":"США", "cm3":"Россия", "hawtai":"Китай", "hummer":"США", "rover":"Англия",
                         "batmobile" : "Россия", "byd": "Китай", "alpina":"Германия", "land rover" : "германия", "уаз":"Россия", "zotye":"Китай", "mercury":"США",
                         "subaru":"Япония", "alfa romeo":"Италия", "proton":"Малазия", "hafei":"Китай", "dacia":"Румыния",
                         "dodge":"США", "газ":"Россия", "landwind":"Китай", "jaguar":"Англия", "тагаз":"Россия", "mini":"Англия",
                         "acura":"Япония", "porsche":"Германия", "lamborghini":"Италия", "daihatsu": "Япония",
                         "cadillac":"США", "москвич":"Россия", "xin kai":"Китай", "volga":"Россия", "иж":"Россия", "saturn":"США",
                         "заз":"Россия", "chery":"Китай", "apal":"Германия", "faw":"Китай", "mercedes-benz":"Германия", "ssangyong":"Корея",
                         "haval":"Китай", "chrysler":"США", 'none': 'none'}
    
    data['mark'] = data['mark'].str.lower()
    
    data['country_mark'] = [countries[i] for i in data['mark'].values]
    
    data = data.fillna(0)
    
    return data
    

In [8]:
df = prepare_data(df)

# Обучение

In [170]:
df_train = prepare_data(df_train)

In [161]:
df_train = pd.read_csv('train_v4.csv')

In [172]:
df_tr_stats = pd.read_csv('stats_all_train.csv')[['order_id', 'sum_fast', 'all_path', 'avg_speed']]
df_tr_stats.columns = ['id_order'] + df_tr_stats.columns.values[1:].tolist()

In [173]:
df_train = pd.merge(df_train, df_tr_stats, on='id_order', how='inner')

In [175]:
df_train['sum_fast/all_path'] = df_train['sum_fast'] / df_train['all_path']

In [182]:
df_train = df_train.drop(['id_order', 'comment_x', 'dt_15_min', 'dt', 'comment_y', 'id_driver', 'id_client'], axis=1)

In [184]:
X = df_train.drop(['is_aggressive'], axis=1)
y = df_train['is_aggressive']

In [190]:
from sklearn.model_selection import cross_val_score

In [234]:
model = catboost.CatBoostClassifier(depth = 2, iterations = 100, l2_leaf_reg = 1, verbose=300, cat_features=['mark', 'country_mark'], auto_class_weights='Balanced', random_seed=2)
model.fit(X, y)
model.save_model('catboost_model.cbm')

0:	learn: 0.6893282	total: 47.4ms	remaining: 4.7s
99:	learn: 0.6190358	total: 1.22s	remaining: 0us


In [232]:
model = catboost.CatBoostClassifier(depth = 2, iterations = 100, l2_leaf_reg = 1, verbose=300, cat_features=['mark', 'country_mark'], auto_class_weights='Balanced', random_seed=2)

np.mean(cross_val_score(model, X, y, scoring='roc_auc', cv=10))

0.7011357137174627


# Усреднение 

In [9]:
#Усредняет все поездки водителя за N дней после заказа


def get_driver_inform(df, order_id, out_params, number_days=7):
    order_day = df[df.id_order == order_id].day.values[0]
    
    driver_id = df[df.id_order == order_id].id_driver.values[0]
    
    order_time = df[df.id_order == order_id].time.values[0]
    
    df = df[(order_time >= df.time) & (order_day - df.day <= number_days) & (df.id_driver == driver_id)]
    
    number_orders = len(df)
    
    df = np.mean(df[out_params])
    
    df = pd.DataFrame([df.values.tolist() + [number_orders]], columns=out_params + ['number_orders'])
    
    return df

In [23]:
#Формирует таблица усреднений за все переданные заказы


def form_clust_data(df, orders, drivers, out_params, catboost_par,  use_transformers=False, use_tracks_info=False):
    
    from_file = catboost.CatBoostClassifier(depth = 4, iterations = 300, l2_leaf_reg = 1, verbose=300, cat_features=['mark', 'country_mark'], auto_class_weights='Balanced')

    from_file.load_model("catboost_model.cbm")
    
    df_stats = pd.read_csv('stats_all.csv')[['all_path', 'avg_speed', 'order_id', 'sum_fast']]
    df_stats.columns = ['all_path', 'avg_speed', 'id_order', 'sum_fast']
    df_stats['sum_fast/all_path'] = df_stats['sum_fast'] / df_stats['all_path']
    
    df = pd.merge(df_stats, df, on='id_order', how='left')
    
    df_predict = df[catboost_par]
    
    df['is_agreesive_proba'] = from_file.predict_proba(df_predict)[:, 1]
    
    df_predict = pd.DataFrame(None)
    
    final_df = pd.DataFrame([], columns=out_params + ['number_orders'])
    
    for order in tqdm(orders):
        final_df = final_df.append(get_driver_inform(df, order, out_params))
        
    final_df['id_order'] = orders
    
    final_df['id_driver'] = drivers
        
    return final_df

In [24]:
out_params = ['duration', 'distance', 'is_agreesive_proba', 'arrived_distance', 'arrived_duration', 'is_rate', 'avg_speed_x', 
             'avg_arrived_speed', 'driver_rate_ride', 'client_rate_ride', 'all_path', 'avg_speed_y', 'sum_fast', 'sum_fast/all_path']

catboost_par = ['arrived_distance', 'arrived_duration', 'distance', 'duration',
       'from_latitude', 'from_longitude', 'to_latitude', 'to_longitude',
       'mark', 'driver_rate_ride', 'client_rate_ride', 'last_total_dist',
       'last_total_time', 'last_total_cnt', 'total_dist', 'total_time',
       'total_cnt', 'cl_last_total_dist', 'cl_last_total_time',
       'cl_last_total_cnt', 'cl_total_dist', 'cl_total_time',
       'cl_total_cnt', 'time', 'day', 'minute', 'month', 'week_day',
       'is_nigth', 'avg_speed_x', 'avg_arrived_speed', 'is_rate',
       'country_mark', 'sum_fast', 'all_path', 'avg_speed_y',
       'sum_fast/all_path']

In [18]:
import catboost

In [13]:
df_april = df[(df.day >= 105) & (df.day <= 118)]
df_april.to_csv('april_data.csv')
new = pd.read_csv('order.csv')

In [30]:
df1 = form_clust_data(df_april, new.order_id.values, new.id_driver.values,  out_params, catboost_par)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 44.59it/s]


In [31]:
def v_rate_coef(numbers):
    return np.array([min(max((30 / i), 0.5), 1.5) for i in numbers])

In [32]:
def k_rate_coef(numbers): 
    return np.array([max(min((i/30), 1.5), 0.5) for i in numbers])

In [35]:
df1['V_rate'] = v_rate_coef(df1.number_orders.values) * (df1.avg_speed_y.values / 60) * df1['sum_fast/all_path'].values * 10 + 0.1

In [36]:
df1.to_csv('V_rate.csv')